# Blast analysis

This notebook aims to find the accessionnumbers of proteins matching our protein data, in order to retrieve their structures from the AlphaFold database.

### Load and filter data

In [ ]:
#Import
import pandas as pd
import numpy as np
import os
import sys
import requests

In [ ]:
#Load the dataframes
A_df = pd.read_csv ('./csv-files/blastp_A.csv', sep = ";")
B_df = pd.read_csv ('./csv-files/blastp_B.csv', sep = ";")
C_df = pd.read_csv ('./csv-files/blastp_C.csv', sep = ";")
D_df = pd.read_csv ('./csv-files/blastp_D.csv', sep = ";")
E_df = pd.read_csv ('./csv-files/blastp_E.csv', sep = ";")
F_df = pd.read_csv ('./csv-files/blastp_F.csv', sep = ";")
G_df = pd.read_csv ('./csv-files/blastp_G.csv', sep = ";")
H_df = pd.read_csv ('./csv-files/blastp_H.csv', sep = ";")
I_df = pd.read_csv ('./csv-files/blastp_I.csv', sep = ";")

#Concatenate into one dataframe and reindex
frames = [A_df, B_df, C_df, D_df, E_df, F_df, G_df, H_df, I_df]
all_df = result = pd.concat(frames)
all_df.reset_index(drop=True, inplace=True)
all_df

In [ ]:
#Drop all rows with less than 80 identity
df = all_df[all_df.pident >= 80]
df = df.sort_values('qseqid')
df.reset_index(drop=True, inplace=True)

#Add columns with %coverage of subject
sstart = df["sstart"]
send = df["send"]
diff = np.subtract(send,sstart)
cover = [(j/i)*100 for i,j in zip(send,diff)]
df["scover"] = cover 

#Drop all rows with less than 80% coverage of subject
df = df[df.scover >= 80]
df = df.sort_values('qseqid')
df.reset_index(drop=True, inplace=True)
df

As seen, some of the proteins have several good matches while others have none. I need to find the best match for the ones with any matches.

### Find best matches

In [ ]:
unique_id = set(df["qseqid"])
print(f"Number of proteins with good matches: {len(unique_id)}")

In [ ]:
#Iterate through the proteins to check matches
accepted_sseqid = dict()
for prot in unique_id:
    subset_df = df[df.qseqid == prot]
    
    #Get best matches on top
    subset_df = subset_df.drop_duplicates(subset=subset_df.columns.difference(['evalue']))
    subset_df = subset_df.sort_values(["pident", "length", "scover"], ascending = (False, False, False))
    
    #Check if there is a drop in values
    id_diff = np.subtract(list(subset_df["pident"])[1:],list(subset_df["pident"])[:-1])
    length_diff = np.subtract(list(subset_df["length"])[1:],list(subset_df["length"])[:-1])
    scover_diff = np.subtract(list(subset_df["scover"])[1:],list(subset_df["scover"])[:-1])
    sseqid = list(subset_df["sseqid"])
    
    #Pick out the best options for structures
    if sum(id_diff) != 0:
        id_idx = [idx for idx, element in enumerate(id_diff) if element != 0]
        id_idx = (id_idx[0])+1
        if id_idx > 5:
            id_idx = 5
        output_sseqid = sseqid[:id_idx]
    elif sum(length_diff) != 0:
        id_idx = [idx for idx, element in enumerate(length_diff) if element != 0]
        id_idx = (id_idx[0])+1
        if id_idx > 5:
            id_idx = 5
        output_sseqid = sseqid[:id_idx]
    elif sum(scover_diff) != 0:
        id_idx = [idx for idx, element in enumerate(scover_diff) if element != 0]
        id_idx = (id_idx[0])+1
        if id_idx > 5:
            id_idx = 5
        output_sseqid = sseqid[:id_idx]
        
    #If all scores are equally good, choose the top 5 (if possible)    
    else:
        output_sseqid = sseqid[:5]
        
    #Save in a dict
    accepted_sseqid[prot] = output_sseqid

    

In [ ]:
print(len(accepted_sseqid))
accepted_sseqid

### Get structures

For easier approach, I choose to only take the first available structure (and thus best if not equal to the others)

In [ ]:
#initialize
total = len(accepted_sseqid)
count = 0

#Iterate through the dict
for key,value in accepted_sseqid.items():
    count += 1
    
    #Screen output
    screen = f"Workign with protein: {key}   {count}/{total}"
    screen = screen.ljust(55, " ")
    print(screen, end = "\r")
    
    #Get the pdb url from alphafold
    best = value[0]
    url = f"https://alphafold.ebi.ac.uk/files/AF-{best}-F1-model_v2.pdb"
    
    #Get pdb access
    try:
        r = requests.get(url, allow_redirects=True)
    except Exception as e:
        print(e)
        sys.exit(1)
        
    #Write the pdb file locally 
    with open(f"pdb-files/{key}_{best}.pdb", "wb") as pdb:
        pdb.write(r.content)